In [3]:
##
## IMPORTS
##

from pathlib import Path
import csv

##
## FUNCTIONS
##

# funtion uses a sniffer to determine whether a header row exists.
def has_header(file_in):
    with open(file_in,'rt',closefd=True) as csv_file:
        # do we have a header?
        sample = csv_file.read(48)
        hh_bool = csv.Sniffer().has_header(sample)
    return hh_bool

##
## MAIN
##

# Set file names for input/output
menu_data_file = Path("./Resources/menu_data.csv")
sales_data_file = Path("./Resources/sales_data.csv")
report_file = Path("./report_metrics.txt")

# initialise lists
menulist = []
saleslist = []

# process MENU data into menulist
with open(menu_data_file,'rt',closefd=True) as csv_file:
    # initialise variables
    csv_generator = csv.reader(csv_file, delimiter=',')

    # skip header if it exists
    if has_header(menu_data_file):
        next(csv_generator)

    # build the list
    for row in csv_generator:
        # iterate through each row and add it the list
        menulist.append(row)

# process SALES data into saleslist
with open(sales_data_file,'rt',closefd=True) as csv_file:
    # initialise variables
    csv_generator = csv.reader(csv_file, delimiter=',')
    
    # skip header if it exists
    if has_header(sales_data_file):
        next(csv_generator)
        
    # build the list
    for row in csv_generator:
        # iterate through each row and add it the list
        saleslist.append(row)

# initialize report dict to hold key/value pairs
report = {}

# Initialize a row counter variable
row_count = 0

# iterate over sales data
for salesrow in saleslist:
    # initialise variables to account for items sold
    # only need Qty and Item, not the rest
    # > Line_Item_ID, Date, Credit_Card_Number, Quantity, Menu_Item
    line_id = int(salesrow[0])
    qty = int(salesrow[3])
    sales_item = salesrow[4]
    
    # initialise the report dict prior to matching against menulist
    # add items for the first time if not currently in report()
    if sales_item not in report.keys():
        report[sales_item] = {"01-count": 0,"02-revenue": 0,"03-cogs": 0,"04-profit": 0}
    
    # walk the list and where there is a match, use the menu data
    # to find price, cost and item
    # > Item, Category, Description, Price, Cost
    for menurow in menulist:
        # initialise variables to perform the match
        # build items from menulist
        menu_item = menurow[0]
        price = float(menurow[3])
        cost = float(menurow[4])
        rev = price*qty
        cogs = cost*qty
        profit = rev-cogs
        
        # for the running tally, start building up the
        # report dict object with key/value pairs
        # note: a guide for understanding values: -
        #   01-count: the total quantity for each ramen type          [quantity]
        #   02-revenue: the total revenue for each ramen type         [revenue = price x quantity]
        #   03-cogs: the total cost of goods sold for each ramen type [cost of goods sold = cost x quantity]
        #   04-profit: the total profit for each ramen type           [profit = revenue less cost]
        # use either salesitem or menu_item as they'll be equal
        if sales_item == menu_item:
            # accumulate values across key metrics for matched item
            report[sales_item]['01-count'] += qty
            report[sales_item]['02-revenue'] += rev
            report[sales_item]['03-cogs'] += cogs
            report[sales_item]['04-profit'] += profit
            
        # !!!!!! ------------------------------------------
        # commenting out these comments here and in
        # the else below, as they cause the .ipynb
        # to be too large to push to git (> 95MB)
        #
        #    # print key values/metrics where matched
        #    #print(f"ln_id[{line_id}] {sales_item} MATCH! > price[{price}] cost[{cost}] qty[{qty}] c_qty[{report[sales_item]['01-count']}] c_rev[{report[sales_item]['02-revenue']}] c_cst[{report[sales_item]['03-cogs']}] c_prf[{report[sales_item]['04-profit']}]")
        # ---------------------------------------------!!!!!
        #else:
        #    # no match for specified item; print that fact
        #    #if row_count <= 1:   ## <- using this to reduce logging while testing
        #        print(f"ln_id[{line_id}] {sales_item} does not equal {menu_item}! NO MATCH!")
    
    # increment counter
    row_count += 1

# processing complete. print total records processed
print("----------------------------------------")    
print(f"Total number of records processed: {row_count}")
print("----------------------------------------")

# write the report dict object to file output
with open(report_file,'w',closefd=True) as fileout:
    for key, value in report.items():
        # for debug - comment out when not debugging
        #print(f"{key} {value}")
        fileout.write(f"{key} {value}\n")


----------------------------------------
Total number of records processed: 74124
----------------------------------------
